In [1]:
import numpy as np
S = np.array([[25., 15., -5.],
              [15., 18., 0.],
              [-5., 0., 11.]])

S

array([[25., 15., -5.],
       [15., 18.,  0.],
       [-5.,  0., 11.]])

In [2]:
L = np.linalg.cholesky(S)
print("Lower triangular matrix L:")
print(L)

Lower triangular matrix L:
[[ 5.  0.  0.]
 [ 3.  3.  0.]
 [-1.  1.  3.]]


In [3]:
import numpy as np

def sigma_to_lz(Sigma):
    """
    Convert a positive definite matrix Sigma into an unconstrained
    parameter vector:
    (l11, z21, z31, ..., zk1, l22, z32, ..., z_k2, ..., lkk)

    where:
        Sigma = L L^T
        L_ii = exp(l_ii)
        L_ij = z_ij for i > j
    """
    Sigma = np.asarray(Sigma)
    k = Sigma.shape[0]

    # Cholesky decomposition (lower triangular)
    L = np.linalg.cholesky(Sigma)

    params = []

    for j in range(k):
        # diagonal: log(L_jj)
        params.append(np.log(L[j, j]))

        # below-diagonal entries in column j
        for i in range(j + 1, k):
            params.append(L[i, j])

    return np.array(params)

In [4]:
params = sigma_to_lz(S)
params

array([ 1.60943791,  3.        , -1.        ,  1.09861229,  1.        ,
        1.09861229])

In [5]:
import numpy as np

def lz_to_sigma(params, k):
    """
    Convert unconstrained parameter vector back to Sigma.

    Parameters
    ----------
    params : array-like, shape (k*(k+1)//2,)
        (l11, z21, z31, ..., zk1, l22, z32, ..., lkk)
    k : int
        Dimension of Sigma

    Returns
    -------
    Sigma : ndarray, shape (k, k)
        Positive definite covariance matrix
    """
    params = np.asarray(params)

    L = np.zeros((k, k))
    idx = 0

    for j in range(k):
        # diagonal
        L[j, j] = np.exp(params[idx])
        idx += 1

        # below diagonal
        for i in range(j + 1, k):
            L[i, j] = params[idx]
            idx += 1

    Sigma = L @ L.T
    return Sigma

In [6]:
lz_to_sigma(params, 3)

array([[ 2.5000000e+01,  1.5000000e+01, -5.0000000e+00],
       [ 1.5000000e+01,  1.8000000e+01,  4.4408921e-16],
       [-5.0000000e+00,  4.4408921e-16,  1.1000000e+01]])

In [7]:
import numpy as np

def lz_to_L(params, k):
    """
    params ordering:
      (l11, z21, z31, ..., zk1, l22, z32, ..., z_k2, ..., lkk)
    where L_ii = exp(l_ii), L_ij = z_ij (i > j).
    """
    params = np.asarray(params, dtype=float)
    L = np.zeros((k, k), dtype=float)

    idx = 0
    for j in range(k):
        L[j, j] = np.exp(params[idx])  # diagonal
        idx += 1
        for i in range(j + 1, k):      # below diagonal in column j
            L[i, j] = params[idx]
            idx += 1
    return L

In [8]:
lz_to_L(params, 3)

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

In [9]:
def jacobian_Sigma_lz(params, k):
    """
    Build Jacobian J = d vec(Sigma) / d params
    
    vec(Sigma) is row-major flattening: Sigma[0,0], Sigma[0,1], ..., Sigma[k-1,k-1]
    
    Returns
    -------
    J : ndarray, shape (k*k, k*(k+1)//2)
    """
    params = np.asarray(params, dtype=float)
    n_params = k * (k + 1) // 2
    if params.size != n_params:
        raise ValueError(f"params length must be {n_params} for k={k}, got {params.size}")

    L = lz_to_L(params, k)
    Sigma = L @ L.T

    J = np.zeros((k * k, n_params), dtype=float)

    # helper: map (i,j) -> row index in vec(Sigma)
    def ridx(i, j):
        return i * k + j

    p = 0
    for j in range(k):
        # --- parameter is l_jj (diagonal log) ---
        a = j
        L_aa = L[a, a]

        # dSigma/ d l_aa
        J[ridx(a, a), p] = 2.0 * (L_aa ** 2)
        for i in range(a + 1, k):
            # (i > a, j = a)
            J[ridx(i, a), p] = L_aa * L[i, a]
            # (i = a, j > a)
            J[ridx(a, i), p] = L_aa * L[i, a]
        p += 1

        # --- parameters are z_{i,j} for i=j+1..k-1 ---
        b = j
        for a in range(j + 1, k):
            # parameter z_ab = L[a,b]
            # dSigma_ij/dz_ab is nonzero only if i=a or j=a
            # Case i=a: dSigma_{a, j}/dz_ab = L[j, b] for j>=b
            for jj in range(b, k):
                J[ridx(a, jj), p] = L[jj, b]
            # Case j=a: dSigma_{i, a}/dz_ab = L[i, b] for i>=b
            for ii in range(b, k):
                J[ridx(ii, a), p] = L[ii, b]
            p += 1

    return J

In [10]:
jacobian_Sigma_lz(params, 3)

array([[50.,  0.,  0.,  0.,  0.,  0.],
       [15.,  5.,  0.,  0.,  0.,  0.],
       [-5.,  0.,  5.,  0.,  0.,  0.],
       [15.,  5.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  0., 18.,  0.,  0.],
       [ 0., -1.,  3.,  3.,  3.,  0.],
       [-5.,  0.,  5.,  0.,  0.,  0.],
       [ 0., -1.,  3.,  3.,  3.,  0.],
       [ 0.,  0., -1.,  0.,  1., 18.]])

In [11]:
S.reshape(-1)

array([25., 15., -5., 15., 18.,  0., -5.,  0., 11.])

In [50]:
def finite_diff_jacobian(params, k, eps=1e-6):
    params = np.asarray(params, dtype=float)
    n = params.size
    base_L = lz_to_L(params, k)
    base_S = base_L @ base_L.T
    base_vec = base_S.reshape(-1)

    Jfd = np.zeros((k*k, n))
    for t in range(n):
        p2 = params.copy()
        p2[t] += eps
        L2 = lz_to_L(p2, k)
        S2 = L2 @ L2.T
        Jfd[:, t] = (S2.reshape(-1) - base_vec) / eps
    return Jfd

In [51]:
finite_diff_jacobian(params, 3, eps=1e-6)

array([[50.00005  ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [15.0000075,  5.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [-5.0000025,  0.       ,  5.       ,  0.       ,  0.       ,
         0.       ],
       [15.0000075,  5.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  6.000001 ,  0.       , 18.000018 ,  0.       ,
         0.       ],
       [ 0.       , -1.       ,  3.       ,  3.0000015,  3.       ,
         0.       ],
       [-5.0000025,  0.       ,  5.       ,  0.       ,  0.       ,
         0.       ],
       [ 0.       , -1.       ,  3.       ,  3.0000015,  3.       ,
         0.       ],
       [ 0.       ,  0.       , -1.999999 ,  0.       ,  2.000001 ,
        18.000018 ]])

In [16]:
def _infer_k_from_nparams(n_params: int) -> int:
    # solve k(k+1)/2 = n_params
    k = int((np.sqrt(8*n_params + 1) - 1) / 2)
    if k * (k + 1) // 2 != n_params:
        raise ValueError(f"invalid n_params={n_params}, cannot infer k")
    return k

def _diag_param_indices(k: int):
    # param order in your lz_to_L: for each column j: [l_jj, z_{j+1,j}, ..., z_{k-1,j}]
    idxs = []
    idx = 0
    for j in range(k):
        idxs.append(idx)      # l_jj
        idx += 1 + (k - 1 - j)
    return idxs

def log_jacobian_lz(params: np.ndarray, k: int) -> float:
    # |dSigma/dparams| = 2^k * Π_j L_jj^{k+1-j} with L_jj = exp(l_jj)
    # => logJac = k log 2 + Σ_j (k+1-j) * l_jj  (j is 0-indexed)
    diag_idxs = _diag_param_indices(k)
    l_diag = params[diag_idxs]
    weights = np.array([k + 1 - j for j in range(k)], dtype=float)
    return float(np.dot(weights, l_diag))


In [17]:
log_jacobian_lz(params, 3)

11.930813093076951

In [23]:
k = 5
rng = np.random.default_rng()   
p = rng.normal(0.0, 1, size=k * (k + 1) // 2)
p

array([-0.52346214,  1.35070421, -1.19322075, -0.20608235, -1.30022767,
       -0.44183564, -2.05050817,  0.06889471,  0.10572005, -1.22284665,
       -1.10853001,  0.53244067, -1.53671521,  1.18685499, -1.05419667])

In [24]:
p @ p.T

17.560112689553208

In [25]:
np.sum((p**2))

17.560112689553208

In [40]:
from scipy.stats import wishart, invwishart, multivariate_normal
def pi(Sigma):
        """
        %pdf of W_1(3, 1) at Sigma
        Sigma can be a scalar or a 1x1 array
        """
        return invwishart.pdf(Sigma, df=3, scale=S)

In [75]:
def grad(Sigma):
    """
    Returns ∇_Σ [ log π(Σ) + (d+1) log|Σ| ] (Euclidean gradient on Sym^+(d)).
    """
    h = 1e-12
    G_2 = np.zeros(Sigma.shape, dtype=float)
    for i in range(3):
        for j in range(3):
            E = np.zeros_like(Sigma)
            E[i, j] = 1.0
            f_plus = np.log(pi(Sigma + h*E))
            f_minus = np.log(pi(Sigma - h*E))
            deriv = (f_plus - f_minus) / (2*h)
            G_2[i, j] = deriv
    #G_2 = 0.5 * (G_2 + G_2.T)
    return G_2

In [76]:
params.size

6

In [77]:
def grad_lz(Sigma):
    """
    Returns ∇_Σ [ log π(Σ) + (d+1) log|Σ| ] (Euclidean gradient on Sym^+(d)).
    """
    h = 1e-6
    para = sigma_to_lz(Sigma)
    print(para)
    G_2 = np.zeros(para.shape, dtype=float)
    for i in range(para.size):
        E = np.zeros_like(para)
        E[i] = 1.0
        print(E)
        S_1 = lz_to_sigma(para + h*E,3)
        print(para + h*E)
        f_plus = np.log(pi(S_1))
        S_2 = lz_to_sigma(para - h*E,3)
        f_minus = np.log(pi(S_2))
        deriv = (f_plus - f_minus) / (2*h)
        print(deriv)
        G_2[i] = deriv
    return G_2

In [78]:
def _infer_k_from_nparams(n_params: int) -> int:
    # solve k(k+1)/2 = n_params
    k = int((np.sqrt(8*n_params + 1) - 1) / 2)
    if k * (k + 1) // 2 != n_params:
        raise ValueError(f"invalid n_params={n_params}, cannot infer k")
    return k
_infer_k_from_nparams(params.size)

3

In [79]:
grad_lz(S)

[ 1.60943791  3.         -1.          1.09861229  1.          1.09861229]
[1. 0. 0. 0. 0. 0.]
[ 1.60943891  3.         -1.          1.09861229  1.          1.09861229]
-5.999999999062311
[0. 1. 0. 0. 0. 0.]
[ 1.60943791  3.000001   -1.          1.09861229  1.          1.09861229]
0.0
[0. 0. 1. 0. 0. 0.]
[ 1.60943791  3.         -0.999999    1.09861229  1.          1.09861229]
0.0
[0. 0. 0. 1. 0. 0.]
[ 1.60943791  3.         -1.          1.09861329  1.          1.09861229]
-5.999999999062311
[0. 0. 0. 0. 1. 0.]
[ 1.60943791  3.         -1.          1.09861229  1.000001    1.09861229]
0.0
[0. 0. 0. 0. 0. 1.]
[ 1.60943791  3.         -1.          1.09861229  1.          1.09861329]
-5.999999999062311


array([-6.,  0.,  0., -6.,  0., -6.])

In [80]:
G = grad(S)
G

array([[-0.28954616, -0.08171241,  0.04440892],
       [ 0.5719869 , -0.36770587, -0.03730349],
       [-0.3126388 ,  0.2593481 , -0.33395509]])

In [81]:
J = jacobian_Sigma_lz(params, 3)
J

array([[50.,  0.,  0.,  0.,  0.,  0.],
       [15.,  5.,  0.,  0.,  0.,  0.],
       [-5.,  0.,  5.,  0.,  0.,  0.],
       [15.,  5.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  0., 18.,  0.,  0.],
       [ 0., -1.,  3.,  3.,  3.,  0.],
       [-5.,  0.,  5.,  0.,  0.,  0.],
       [ 0., -1.,  3.,  3.,  3.,  0.],
       [ 0.,  0., -1.,  0.,  1., 18.]])

In [82]:
g_sigma_vec = G.reshape(-1, order="C")
g_sigma_vec

array([-0.28954616, -0.08171241,  0.04440892,  0.5719869 , -0.36770587,
       -0.03730349, -0.3126388 ,  0.2593481 , -0.33395509])

In [84]:
grad = J.T @ g_sigma_vec

In [92]:
# 4. Jacobian correction term
jac_corr = np.zeros_like(grad)
k = 3
j = 1
for idx in diag_param_indices(3):
    # diagonal parameter l_jj
    jac_corr[idx] = k + 1 - j   # i = j+1 in math indexing
    j = j+1
    
jac_corr

array([3., 0., 0., 2., 0., 1.])

In [88]:
def diag_param_indices(k: int) -> np.ndarray:
    """
    params ordering: (l11, z21, z31, ..., lk1, l22, z32, ..., lkk)
    Returns indices of l_jj entries in the params vector.
    """
    idxs = []
    idx = 0
    for j in range(k):
        idxs.append(idx)                 # l_jj
        idx += 1 + (k - 1 - j)          # skip l_jj + all z below it in column j
    return np.array(idxs, dtype=int)
diag_param_indices(3)

array([0, 3, 5])

In [ ]:
import numpy as np

def grad_theta(G, params):
    """
    Compute gradient d log p(l,z | data) / d theta

    Parameters
    ----------
    gradient : ndarray (k, k)
        d log p(Sigma | data) / d Sigma
    params : ndarray (k*(k+1)//2,)
        (l11, z21, ..., lkk)
    k : int

    Returns
    -------
    grad_theta : ndarray (k*(k+1)//2,)
    """
    # 1. Jacobian of vec(Sigma) wrt theta
    k = gradient.shape[0]
    J = jacobian_Sigma_lz(params, k)

    # 2. vec(G)
    vecG = G.reshape(-1)   # row-major

    # 3. chain rule term
    grad = J.T @ vecG

    # 4. Jacobian correction term
    jac_corr = np.zeros_like(grad)

    idx = 0
    for j in range(k):
        # diagonal parameter l_jj
        jac_corr[idx] = k + 2 - (j + 1)   # i = j+1 in math indexing
        idx += 1

        # off-diagonal z_ij → 0
        for _ in range(j + 1, k):
            idx += 1

    return grad + jac_corr